In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy
from scipy import stats

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# sys.path.append(module_path)

# from utils.wheel import wheel
# from utils.calcium import calcium
# from utils.animal_database import animal_database


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<ipython-input-2-7738a0d6c81d>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [39]:

# FUNCTION FOR NATHALIE'S MERGEING RUNNING and NONTMOVING BOUTS

def replace_sequences_in_place(arr, n):

    #
    idx = np.where(arr==1)[0]
    idx_diff = np.diff(idx)

    # fill in all sequences less than n with 0s
    for i in range(len(idx_diff)):
        if idx_diff[i]<n:
            arr[idx[i]:idx[i+1]] = 1
 
    idx = np.where(arr==0)[0]
    idx_diff = np.diff(idx)

    # fill in all sequences less than n with 0s
    for i in range(len(idx_diff)):
        if idx_diff[i]<n:
            arr[idx[i]:idx[i+1]] = 0
 
    return arr
    

    #


# make a boolean array of all stationary frames 
idx_stationary = np.array([0,0,0,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,0,1,0,0])
print (idx_stationary)

idx_stationary = replace_sequences_in_place(idx_stationary, 5)
print (idx_stationary)



[0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0]
diffs:  [1 1 1 1 1 1 3 1 6]
diffs:  [ 1  1 12  1  1  1  1  2  1]
[0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]


In [10]:
#############################################
########### LOAD ALREADY COMPUTED PCA #######
#############################################
root_dir = '/media/cat/4TB/donato/'
binarization_method = 'upphase'
quiescent_periods = True
cell_randomization = False
recompute = False

#                          #cells, #secs,  #P-day start
animal_ids = {'DON-006084':[200, 600, 19], 
              'DON-006085':[500, 600, 19],
              'DON-003165':[200, 600, 120],
              'DON-003343':[400, 600, 150]
             }


evs = []
times = []
for kk in animal_ids.items():
    
    # 
    animal_id = kk[0]
    
    # 
    n_cells = kk[1][0]
    n_times = kk[1][1]
    p_start = kk[1][2]
    # 
    ad = animal_database.AnimalDatabase()
    ad.load_sessions(animal_id)

    # 
    animal_id = kk[0]
    
    # 
    n_cells = kk[1][0]
    n_times = kk[1][1]
    p_start = kk[1][2]# 
    clrs = ['black','blue','red']

    # 
    binarization_method = 'upphase'
    cum_var = []
    singular_vals = []

    #n_cells = 400
    #n_times = 600
    temp_times = []
    temp_evs = []
    for ctr, session in enumerate(ad.sessions):

        # 
        c = calcium.Calcium()
        c.root_dir = root_dir
        c.data_dir = os.path.join(root_dir, animal_id, session)
        c.animal_id = animal_id
        c.session = session
        c.load_suite2p()

        #c.load_binarization()

        #
        try:
            pca, X_pca = c.load_PCA(session, n_cells, n_times)
        except:
            print ("Pca not available; ", animal_id, session)
            continue
        #
        temp_times.append(ctr+p_start)
        ctr+=1

        #print (session, X_pca.shape)
        cum_var.append(pca.explained_variance_ratio_)

        #
        temp_evs.append(pca.singular_values_)
    
    times.append(temp_times)
    evs.append(temp_evs)

    # 
    print ("# of sessions loaded: ", animal_id, len(cum_var))

Final cumulative variance explained:  10
Pca not available;  DON-006085 20210521
Final cumulative variance explained:  9
Pca not available;  DON-003165 DON-003165_20210215
Pca not available;  DON-003165 DON-003165_20210219
Final cumulative variance explained:  8
Pca not available;  DON-003343 DON-003343_20210215
Pca not available;  DON-003343 DON-003343_20210221
Pca not available;  DON-003343 DON-003343_20210222
Final cumulative variance explained:  7


In [13]:
#######################################################################
###### LINEAR REGRESSION EIGENVALUE SPECTRUM/DECAY - ALL SESSIONS #####
#######################################################################
clrs = ['black','blue','red','green']

fig=plt.figure(figsize=(10,5))
ctr=0
for times1, evs1 in zip(times, evs):
    
    slopes = []
    singular_vals = evs1
    for k in range(len(singular_vals)):

        # get eigenvalues
        temp = singular_vals[k]

        # take log
        temp_log = np.log(temp)

        # 
        x = np.arange(len(temp_log))
        z = np.polyfit(x, temp_log, 1)
        slopes.append(z[0])
        z = np.poly1d(z)(x)
    

    # 
    x = times1
    z = np.polyfit(x, slopes, 1)
    z = np.poly1d(z)(x)
    
    #
    pcor = stats.pearsonr(x,slopes)

    #
    plt.scatter(times1, 
                slopes, 
                c=clrs[ctr],
               label=list(animal_ids.keys())[ctr]+": p-cor:"+str(round(pcor[0],3)).zfill(4)+", p-val:"+str(round(pcor[1],3)))
    if pcor[1]<0.05:
        plt.plot(x,z,c=clrs[ctr])
    else:
        plt.plot(x,z,'--',c=clrs[ctr],alpha=.5)

    ctr+=1
    
plt.ylim(top=0)
plt.legend()
plt.title("Trends in Singular Value Decay (PCA) of Population Dynamics During Quiescent Periods"+
         "\n (0-slope: completely correlated activity; -1 slope: all noise)")
plt.xlabel("Post-natal Day")
plt.ylabel('Slope of Log(Eigenvalue Spectrum Decay)')
plt.show()